# Air-Source Heat Pump Module

Models reversible air-source heat pump (ASHP) for both heating and cooling. COP varies with temperature lift, demonstrating superior exergy efficiency compared to direct fuel combustion.

## Heating Mode - Energy Analysis

**Temperature lift:**
$$\Delta T = T_{sink} - T_{source}$$

**COP correlation (heating):**
$$COP_{heating} = 6.08 - 0.0941 \cdot \Delta T + 0.000464 \cdot \Delta T^2$$

**Heat output:**
$$\dot{Q}_{heating} = COP_{heating} \cdot \dot{W}_{elec}$$

**Energy efficiency:**
$$\eta_{en,HP} = COP_{heating} > 1$$

## Heating Mode - Exergy Analysis

**Electrical exergy input:**
$$\dot{Ex}_{in} = \dot{W}_{elec}$$

**Thermal exergy output:**
$$\dot{Ex}_{heat,out} = \dot{Q}_{heating} \left(1 - \frac{T_0}{T_{sink}}\right)$$

**Exergy efficiency:**
$$\Psi_{HP,heat} = \frac{\dot{Ex}_{heat,out}}{\dot{W}_{elec}} = COP_{heating} \cdot \left(1 - \frac{T_0}{T_{sink}}\right)$$

Typical: $\Psi_{HP} \approx 40-50\%$ (vs 5-7% for furnace!)

## Cooling Mode - Energy Analysis

**Temperature lift (cooling):**
$$\Delta T_{cool} = T_{ambient} - T_{indoor}$$

**COP correlation (cooling):**
$$COP_{cooling} = 5.08 - 0.0941 \cdot \Delta T_{cool} + 0.000464 \cdot \Delta T_{cool}^2$$

**Cooling output:**
$$\dot{Q}_{cooling} = COP_{cooling} \cdot \dot{W}_{elec}$$

## Cooling Mode - Exergy Analysis

**Cooling exergy output:**
$$\dot{Ex}_{cool,out} = \dot{Q}_{cooling} \left(\frac{T_0}{T_{indoor}} - 1\right)$$

**Exergy efficiency:**
$$\Psi_{HP,cool} = \frac{\dot{Ex}_{cool,out}}{\dot{W}_{elec}} = COP_{cooling} \cdot \left(\frac{T_0}{T_{indoor}} - 1\right)$$

## Parameters

In [1]:
# Heat Pump Default Parameters
const HP_DEFAULTS = (
    # Heating COP coefficients: COP = a - b*ΔT + c*ΔT²
    a_heat = 6.08,
    b_heat = 0.0941,
    c_heat = 0.000464,
    # Cooling COP coefficients
    a_cool = 5.08,
    b_cool = 0.0941,
    c_cool = 0.000464,
    # Default temperatures
    T_sink = 308.15,      # Indoor heating temp [K] (35°C for floor heating)
    T_indoor = 298.15,    # Indoor cooling temp [K] (25°C)
    COP_min = 1.0         # Minimum COP limit
)

(a_heat = 6.08, b_heat = 0.0941, c_heat = 0.000464, a_cool = 5.08, b_cool = 0.0941, c_cool = 0.000464, T_sink = 308.15, T_indoor = 298.15, COP_min = 1.0)

## Core Functions

In [2]:
"""
    HeatPump

Struct holding air-source heat pump parameters.
"""
struct HeatPump
    Q_rated_heat::Float64   # Rated heating capacity [W]
    Q_rated_cool::Float64   # Rated cooling capacity [W]
    # Heating COP coefficients
    a_heat::Float64
    b_heat::Float64
    c_heat::Float64
    # Cooling COP coefficients
    a_cool::Float64
    b_cool::Float64
    c_cool::Float64
    # Temperature setpoints
    T_sink::Float64         # Heating delivery temp [K]
    T_indoor::Float64       # Cooling indoor temp [K]
    COP_min::Float64        # Minimum COP
    
    function HeatPump(Q_rated_heat::Float64, Q_rated_cool::Float64;
            a_heat::Float64 = 6.08, b_heat::Float64 = 0.0941, c_heat::Float64 = 0.000464,
            a_cool::Float64 = 5.08, b_cool::Float64 = 0.0941, c_cool::Float64 = 0.000464,
            T_sink::Float64 = 308.15, T_indoor::Float64 = 298.15,
            COP_min::Float64 = 1.0)
        new(Q_rated_heat, Q_rated_cool, a_heat, b_heat, c_heat,
            a_cool, b_cool, c_cool, T_sink, T_indoor, COP_min)
    end
end

# Constructor with capacity in kW
function HeatPump(; Q_heat_kW::Float64, Q_cool_kW::Float64, kwargs...)
    HeatPump(Q_heat_kW * 1000.0, Q_cool_kW * 1000.0; kwargs...)
end

HeatPump

In [3]:
"""
    cop_heating(hp::HeatPump, T_source::Float64) -> Float64

Calculate heating COP at given source (outdoor) temperature [K].
"""
function cop_heating(hp::HeatPump, T_source::Float64)
    ΔT = hp.T_sink - T_source
    COP = hp.a_heat - hp.b_heat * ΔT + hp.c_heat * ΔT^2
    return max(COP, hp.COP_min)
end

cop_heating

In [4]:
"""
    cop_cooling(hp::HeatPump, T_ambient::Float64) -> Float64

Calculate cooling COP at given ambient (outdoor) temperature [K].
"""
function cop_cooling(hp::HeatPump, T_ambient::Float64)
    ΔT = T_ambient - hp.T_indoor
    COP = hp.a_cool - hp.b_cool * ΔT + hp.c_cool * ΔT^2
    return max(COP, hp.COP_min)
end

cop_cooling

In [5]:
"""
    heating_power(hp::HeatPump, Q_heating::Float64, T_source::Float64) -> Float64

Calculate electrical power [W] required for given heating output [W].
"""
function heating_power(hp::HeatPump, Q_heating::Float64, T_source::Float64)
    COP = cop_heating(hp, T_source)
    return Q_heating / COP
end

heating_power

In [6]:
"""
    cooling_power(hp::HeatPump, Q_cooling::Float64, T_ambient::Float64) -> Float64

Calculate electrical power [W] required for given cooling output [W].
"""
function cooling_power(hp::HeatPump, Q_cooling::Float64, T_ambient::Float64)
    COP = cop_cooling(hp, T_ambient)
    return Q_cooling / COP
end

cooling_power

In [7]:
"""
    analyze_heating(hp::HeatPump, Q_heating::Float64, T_source::Float64, T_0::Float64) -> NamedTuple

Complete heating mode analysis.
Q_heating: heat demand [W], T_source: outdoor temp [K], T_0: dead state temp [K]
"""
function analyze_heating(hp::HeatPump, Q_heating::Float64, T_source::Float64, T_0::Float64)
    @assert Q_heating >= 0 "Heat demand must be non-negative"
    @assert Q_heating <= hp.Q_rated_heat "Exceeds rated heating capacity"
    
    # Energy analysis
    COP = cop_heating(hp, T_source)
    W_elec = Q_heating / COP
    
    # Exergy analysis
    Ex_in = W_elec  # Electricity is pure exergy
    η_carnot = 1.0 - T_0 / hp.T_sink
    Ex_heat = Q_heating * η_carnot
    Ex_d = Ex_in - Ex_heat
    Ψ = Ex_heat / Ex_in
    
    return (
        mode = :heating,
        Q_thermal = Q_heating,    # Thermal output [W]
        W_elec = W_elec,          # Electrical input [W]
        COP = COP,                # Coefficient of Performance [-]
        T_source = T_source,      # Source temperature [K]
        T_sink = hp.T_sink,       # Sink temperature [K]
        T_0 = T_0,                # Dead state temperature [K]
        ΔT = hp.T_sink - T_source,# Temperature lift [K]
        Ex_in = Ex_in,            # Exergy input [W]
        Ex_out = Ex_heat,         # Exergy output [W]
        Ex_d = Ex_d,              # Exergy destruction [W]
        η_carnot = η_carnot,      # Carnot factor [-]
        Ψ = Ψ                     # Exergy efficiency [-]
    )
end

analyze_heating

In [8]:
"""
    analyze_cooling(hp::HeatPump, Q_cooling::Float64, T_ambient::Float64, T_0::Float64) -> NamedTuple

Complete cooling mode analysis.
Q_cooling: cooling demand [W], T_ambient: outdoor temp [K], T_0: dead state temp [K]
"""
function analyze_cooling(hp::HeatPump, Q_cooling::Float64, T_ambient::Float64, T_0::Float64)
    @assert Q_cooling >= 0 "Cooling demand must be non-negative"
    @assert Q_cooling <= hp.Q_rated_cool "Exceeds rated cooling capacity"
    
    # Energy analysis
    COP = cop_cooling(hp, T_ambient)
    W_elec = Q_cooling / COP
    
    # Exergy analysis
    # For cooling: exergy = Q * (T_0/T_cold - 1) where T_cold < T_0
    Ex_in = W_elec
    carnot_cool = T_0 / hp.T_indoor - 1.0  # Positive when T_indoor < T_0
    Ex_cool = Q_cooling * carnot_cool
    Ex_d = Ex_in - Ex_cool
    Ψ = Ex_cool / Ex_in
    
    return (
        mode = :cooling,
        Q_thermal = Q_cooling,    # Thermal output [W]
        W_elec = W_elec,          # Electrical input [W]
        COP = COP,                # Coefficient of Performance [-]
        T_ambient = T_ambient,    # Ambient temperature [K]
        T_indoor = hp.T_indoor,   # Indoor temperature [K]
        T_0 = T_0,                # Dead state temperature [K]
        ΔT = T_ambient - hp.T_indoor,  # Temperature lift [K]
        Ex_in = Ex_in,            # Exergy input [W]
        Ex_out = Ex_cool,         # Exergy output [W]
        Ex_d = Ex_d,              # Exergy destruction [W]
        carnot_cool = carnot_cool,# Cooling Carnot factor [-]
        Ψ = Ψ                     # Exergy efficiency [-]
    )
end

analyze_cooling

## Tests

In [9]:
using Test

function run_hp_tests()
    println("Running Heat Pump Tests...")
    
    # Create test heat pump: 10 kW heating, 8 kW cooling
    hp = HeatPump(Q_heat_kW=10.0, Q_cool_kW=8.0, T_sink=308.15, T_indoor=298.15)
    
    # Test 1: Parameters
    @testset "Parameters" begin
        @test hp.Q_rated_heat == 10000.0
        @test hp.Q_rated_cool == 8000.0
        @test hp.T_sink == 308.15  # 35°C
        @test hp.T_indoor == 298.15  # 25°C
    end
    
    # Test 2: Heating COP
    @testset "Heating COP" begin
        # At mild conditions (ΔT small), COP should be high
        COP_mild = cop_heating(hp, 283.15)  # 10°C outdoor
        COP_cold = cop_heating(hp, 263.15)  # -10°C outdoor
        @test COP_mild > COP_cold  # COP drops as it gets colder
        @test COP_mild > 1.0
        @test COP_cold > 1.0
    end
    
    # Test 3: Cooling COP
    @testset "Cooling COP" begin
        # At mild conditions (ΔT small), COP should be high
        COP_mild = cop_cooling(hp, 303.15)  # 30°C outdoor
        COP_hot = cop_cooling(hp, 313.15)   # 40°C outdoor
        @test COP_mild > COP_hot  # COP drops as it gets hotter
        @test COP_mild > 1.0
        @test COP_hot > 1.0
    end
    
    # Test 4: Heating power calculation
    @testset "Heating Power" begin
        Q = 5000.0  # 5 kW heating
        T_source = 273.15  # 0°C
        W = heating_power(hp, Q, T_source)
        COP = cop_heating(hp, T_source)
        @test W ≈ Q / COP atol=1e-6
        @test W < Q  # Electrical input < thermal output (COP > 1)
    end
    
    # Test 5: Heating exergy analysis
    @testset "Heating Exergy" begin
        T_source = 273.15  # 0°C outdoor
        T_0 = 273.15       # Dead state = outdoor
        result = analyze_heating(hp, 5000.0, T_source, T_0)
        
        # Exergy balance
        @test result.Ex_in ≈ result.Ex_out + result.Ex_d atol=1e-6
        # COP relationship
        @test result.COP ≈ result.Q_thermal / result.W_elec atol=1e-6
        # Exergy efficiency relationship
        @test result.Ψ ≈ result.COP * result.η_carnot atol=1e-6
        # Exergy efficiency should be reasonable (30-60%)
        @test 0.2 < result.Ψ < 0.8
    end
    
    # Test 6: Cooling exergy analysis
    @testset "Cooling Exergy" begin
        T_ambient = 308.15  # 35°C outdoor
        T_0 = 308.15        # Dead state = outdoor
        result = analyze_cooling(hp, 4000.0, T_ambient, T_0)
        
        # Exergy balance
        @test result.Ex_in ≈ result.Ex_out + result.Ex_d atol=1e-6
        # Cooling exergy should be positive (T_indoor < T_0)
        @test result.Ex_out > 0
        # COP relationship
        @test result.COP ≈ result.Q_thermal / result.W_elec atol=1e-6
    end
    
    # Test 7: Compare with Carnot
    @testset "Carnot Comparison" begin
        T_source = 273.15
        COP_actual = cop_heating(hp, T_source)
        COP_carnot = hp.T_sink / (hp.T_sink - T_source)
        @test COP_actual < COP_carnot  # Real COP < Carnot COP
    end
    
    println("All tests passed!")
end

run_hp_tests()

Running Heat Pump Tests...
Test Summary: | Pass  Total  Time
Parameters    |    4      4  0.0s
Test Summary: | Pass  Total  Time
Heating COP   |    3      3  0.0s
Test Summary: | Pass  Total  Time
Cooling COP   |    3      3  0.0s
Test Summary: | Pass  Total  Time
Heating Power |    2      2  0.1s
Test Summary:  | Pass  Total  Time
Heating Exergy |    4      4  0.0s
Test Summary:  | Pass  Total  Time
Cooling Exergy |    3      3  0.0s
Test Summary:     | Pass  Total  Time
Carnot Comparison |    1      1  0.0s
All tests passed!


## Example Usage

In [10]:
# Example: 10 kW heat pump
hp = HeatPump(Q_heat_kW=10.0, Q_cool_kW=8.0, T_sink=308.15, T_indoor=298.15)

# Heating analysis at 0°C outdoor
result_heat = analyze_heating(hp, 8000.0, 273.15, 273.15)

println("Heat Pump - HEATING Mode")
println("="^50)
println("Heat demand:        $(result_heat.Q_thermal/1000) kW")
println("Outdoor temp:       $(round(result_heat.T_source - 273.15, digits=1)) °C")
println("Delivery temp:      $(round(result_heat.T_sink - 273.15, digits=1)) °C")
println("Temperature lift:   $(round(result_heat.ΔT, digits=1)) K")
println("─"^50)
println("Electrical input:   $(round(result_heat.W_elec/1000, digits=2)) kW")
println("COP:                $(round(result_heat.COP, digits=2))")
println("─"^50)
println("Exergy input:       $(round(result_heat.Ex_in/1000, digits=2)) kW")
println("Exergy output:      $(round(result_heat.Ex_out/1000, digits=2)) kW")
println("Exergy destruction: $(round(result_heat.Ex_d/1000, digits=2)) kW")
println("Carnot factor:      $(round(result_heat.η_carnot*100, digits=1))%")
println("Exergy efficiency:  $(round(result_heat.Ψ*100, digits=1))%")

Heat Pump - HEATING Mode
Heat demand:        8.0 kW
Outdoor temp:       0.0 °C
Delivery temp:      35.0 °C
Temperature lift:   35.0 K
──────────────────────────────────────────────────
Electrical input:   2.38 kW
COP:                3.35
──────────────────────────────────────────────────
Exergy input:       2.38 kW
Exergy output:      0.91 kW
Exergy destruction: 1.48 kW
Carnot factor:      11.4%
Exergy efficiency:  38.1%


In [11]:
# Cooling analysis at 35°C outdoor
result_cool = analyze_cooling(hp, 6000.0, 308.15, 308.15)

println("\nHeat Pump - COOLING Mode")
println("="^50)
println("Cooling demand:     $(result_cool.Q_thermal/1000) kW")
println("Outdoor temp:       $(round(result_cool.T_ambient - 273.15, digits=1)) °C")
println("Indoor temp:        $(round(result_cool.T_indoor - 273.15, digits=1)) °C")
println("Temperature lift:   $(round(result_cool.ΔT, digits=1)) K")
println("─"^50)
println("Electrical input:   $(round(result_cool.W_elec/1000, digits=2)) kW")
println("COP:                $(round(result_cool.COP, digits=2))")
println("─"^50)
println("Exergy input:       $(round(result_cool.Ex_in/1000, digits=2)) kW")
println("Exergy output:      $(round(result_cool.Ex_out/1000, digits=2)) kW")
println("Exergy destruction: $(round(result_cool.Ex_d/1000, digits=2)) kW")
println("Cooling Carnot:     $(round(result_cool.carnot_cool*100, digits=1))%")
println("Exergy efficiency:  $(round(result_cool.Ψ*100, digits=1))%")


Heat Pump - COOLING Mode
Cooling demand:     6.0 kW
Outdoor temp:       35.0 °C
Indoor temp:        25.0 °C
Temperature lift:   10.0 K
──────────────────────────────────────────────────
Electrical input:   1.43 kW
COP:                4.19
──────────────────────────────────────────────────
Exergy input:       1.43 kW
Exergy output:      0.2 kW
Exergy destruction: 1.23 kW
Cooling Carnot:     3.4%
Exergy efficiency:  14.0%


In [ ]:
# COP vs outdoor temperature (wrapped in let block for NBInclude compatibility)
let
    println("\nHeating COP vs Outdoor Temperature")
    println("="^40)
    println("T_out [°C]    ΔT [K]    COP    Ψ [%]")
    println("─"^40)

    for T_out_C in [-15, -10, -5, 0, 5, 10, 15]
        T_source = T_out_C + 273.15
        T_0 = T_source  # Dead state = outdoor
        result = analyze_heating(hp, 5000.0, T_source, T_0)
        println("$(lpad(T_out_C, 8))    $(lpad(round(result.ΔT, digits=0), 6))    $(lpad(round(result.COP, digits=2), 5))    $(lpad(round(result.Ψ*100, digits=1), 5))")
    end
end

## Export

Functions available for import:
- `HeatPump` - Model struct
- `cop_heating` - Heating COP at given outdoor temp
- `cop_cooling` - Cooling COP at given outdoor temp
- `heating_power` - Electrical power for heating
- `cooling_power` - Electrical power for cooling
- `analyze_heating` - Complete heating analysis
- `analyze_cooling` - Complete cooling analysis